In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch
from torch import nn
%matplotlib inline
import matplotlib.pyplot as plt
from dnaDataloader import expermentDataloader
from dnaDataloader import addData
from dnaModelUtil import MLPModel
from dnaModelUtil import train
from dnaModelUtil import progress_bar
from datetime import datetime
import json
import pandas as pd
import seaborn as sns
defultePlotSize = plt.rcParams['figure.figsize']
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
folder = '/home/khood/GitHub/SNN-DNA-project/Prepocessing/sorted/1800_nM_AR_5000'
oneMinInFPS = 1200
batch_size = 10
num_workers = 0
device

device(type='cuda', index=2)

In [17]:
oneMinInFPS*((1/oneMinInFPS)*10)

10.0

## making graphs for short movie length

In [1]:
def makeDatasets(length: int, folder: str):
    length = int(length*oneMinInFPS)
    assert length > 0
    data = expermentDataloader(
        f"{folder}/index.csv",
        f"{folder}",
        length=length
    )
    rawData = [d for d in data]
    trainValidData = []
    testData = []
    addData(testData, trainValidData, rawData, rhsSize=300)

    np.random.shuffle(trainValidData)
    trainData = []
    validData = []
    addData(trainData, validData, trainValidData,
            rhsSize=int(len(trainValidData)*(1/3)))

    trainDataset = DataLoader(trainData, batch_size=batch_size,
                              shuffle=True, num_workers=num_workers, pin_memory=True)
    validDataset = DataLoader(validData, batch_size=batch_size,
                              shuffle=True, num_workers=num_workers, pin_memory=True)
    testDataset = DataLoader(testData, batch_size=len(
        testData), shuffle=True, num_workers=num_workers, pin_memory=True)
    # print(f"Training dataset size: {len(trainData)}")
    # print(f"Valid dataset size: {len(validData)}")
    # print(f"Testing dataset size: {len(testData)}")
    # print(f"Total: {len(trainData) + len(validData) + len(testData)}")
    # print(f"number train batches:{len(trainDataset)}")
    # print(f"number valid batches:{len(validDataset)}")
    # print(f"number test batches:{len(testDataset)}")
    return trainDataset, validDataset, testDataset

In [9]:
datasests = {}
maxLength = 3*oneMinInFPS
minLength = 1/oneMinInFPS
step = (1/oneMinInFPS)*10000000
total = len(np.arange(minLength, maxLength+step, step))
total

2

In [5]:
datasests = {}
current = 0
for length in np.arange(minLength, maxLength+step, step):
    datasests[f"{length}"] = makeDatasets(length, folder)
    current += 1
    progress_bar(current=current, total=total, bar_length=20)

Progress: [------------------->] 100%


In [ ]:
datasests

In [ ]:
input, target = next(iter(datasests["0.0008333333333333334"][0]))
target

In [ ]:
val_data = datasests[f"{minLength}"][1]
train_data = datasests[f"{minLength}"][0]
print(f"val_data: {val_data}")
print(f"train_data: {train_data}")

In [ ]:
len(next(iter(train_data))[0][0])

In [ ]:

model_dir = "/home/khood/GitHub/SNN-DNA-project/Models/modelsForVariedLengthed"
em = 5
epochs = 10
# train(
# trainData: DataLoader, validData: DataLoader, name: str, savePath: str,
# model, lossfunction, optim, return_dict: dict, epochs: int, margin_of_error=20, device=torch.device("cpu"), printStatus=False)
resultes = {}
return_dict = {}
current = 0
keys = list(datasests.keys())
while keys:
    length = float(keys.pop(0))
    val_data = datasests[f"{length}"][1]
    train_data = datasests[f"{length}"][0]
    # next(iter(train_data))
    featIn = len(next(iter(train_data))[0][0])
    # print(f"featIn: {featIn}")
    model = MLPModel(featIn=int(featIn), capacity=1000)
    model = model.to(device)
    MSE = nn.MSELoss(reduction='mean')
    adam = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=1e-5)
    resultes[f"{length}"] = train(trainData=train_data, validData=val_data, name=f"{length}_min(s)", savePath=model_dir, margin_of_error=em,
                                model=model, lossfunction=MSE, optim=adam, return_dict=return_dict, epochs=600, device=device, printStatus=False)
    current += 1
    progress_bar(current=current, total=total, bar_length=20)

In [ ]:
resultes[list(resultes.keys())[0]].keys()

In [ ]:
header = []
acc_data = []
loss_data = []
for min in resultes.keys():
    header.append(min)
    acc_data.append(resultes[min]["accuracy_val"])
    loss_data.append(resultes[min]["loss_val"])

acc_data_df = pd.DataFrame(acc_data, index=header).T
loss_data = pd.DataFrame(loss_data, index=header).T

In [ ]:
acc_data_df

In [ ]:
plt.rcParams['figure.figsize'] = [60, 25]
sns.set(font_scale=2)
ax = sns.lineplot(data=acc_data_df)
sns.move_legend(
    ax, "lower center",
    bbox_to_anchor=(.5, 1), ncol=10, title="accuracy vs movie length 1-20 minutes", frameon=False,
)
fig = plt.figure()
fig.subplots_adjust(hspace=0.5, wspace=0.1)
ax.set(xlabel="epochs", ylabel="accuracy")
vals = ax.get_yticks()
ax.set_yticklabels(['{:,.1%}'.format(x) for x in vals])
# ax.set(title='accuracy vs movie length 1-10 minutes')

In [ ]:
max_acc = pd.DataFrame(acc_data_df.max()).T
max_acc

In [ ]:
# names = list(sorted_value_rnn_valid_acc.keys())
# values = list(sorted_value_rnn_valid_acc.values())

plt.rcParams['figure.figsize'] = [30, 15]
a = sns.barplot(data=max_acc)
a.set(xlabel="length in seconds", ylabel="accuracy")
vals = a.get_yticks()
a.set_yticklabels(['{:,.1%}'.format(x) for x in vals])
vals = a.get_xticklabels()
a.set_xticklabels(['{:,.1}'.format(float(x.get_text())) for x in vals])
a.set(title='accuracy vs movie length 1-20 minutes')

In [ ]:
vals

In [ ]:
maxLength

In [ ]:
with open('/home/khood/GitHub/SNN-DNA-project/Models/mlp_1frame_1min_acc.json', "w") as f:
    json.dump(resultes, f)

In [ ]:
plt.rcParams['figure.figsize'] = [40, 500]
graph = 1
fig = plt.figure()

fig.subplots_adjust(hspace=0.5, wspace=0.3)
# fig.tight_layout()
j = 1
for key in list(resultes.keys())[:1]:
    plt.subplot(2*j, 2, graph)
    plt.title(f'{key} min(s) accuracy vs epochs training')
    plt.xlabel("epochs")
    plt.ylabel("accuracy")
    plt.plot(list(range(len(resultes[f"{key}"]["accuracy_val"][:]))),
             resultes[f"{key}"]["accuracy_val"][:], '-', color='black', )
    graph += 1
    plt.subplot(2*j, 2, graph)
    plt.title(f'{key} min(s) losse vs epochs training')
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.plot(list(range(len(resultes[f"{key}"]["loss_val"])))[
             :], resultes[f"{key}"]["loss_val"][:], '-', color='black')
    graph += 1
    j += 1
# plt.suptitle("test")
plt.rcParams['figure.figsize'] = defultePlotSize